In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm

# Stability of least squares algorithms

Let's set up a least squares problem.  We'll fit a polynomial of degree 14 to the function 

$$e^{\sin(4t)}.$$

We'll normalize the function (based on knowledge of the exact answer) so that the correct answer for the leading coefficient of the interpolating polynomial is 1.

In [ ]:
m = 100  # Sample at this many points
n = 15   # Fit a polynomial of degree n-1

t = np.linspace(0,1,m)
A = np.zeros((m,n))
for i in range(n):
    A[:,i] = t**i      # Vandermonde matrix
    
b = np.exp(np.sin(4*t))  # Sampled function
b = b/2006.787453080206  # Normalization
plt.plot(b);

Now let's determine the least squares condition number for this problem.  Note that in order to do so, we have to actually solve the problem.  We also plot the fit, which is very close.

In [ ]:
x, res, rank, s = np.linalg.lstsq(A,b,rcond=-1)
y = np.dot(A,x)
kappa = np.linalg.cond(A)
tantheta = norm(b-y)/norm(y)
eta = norm(A)*norm(x)/norm(y)
term2 = kappa**2 * tantheta/eta
print('kappa: %0.2e' % kappa)
print('theta: %0.2e' % np.arctan(tantheta))
print('eta: %10.2e' % eta)
print('kappa^2 * tan(theta)/eta: %10.2e' % term2)

plt.plot(t,b,'-',t,y,'o'); plt.xlabel('x'); plt.ylabel('$f(x)$');

We see that the matrix is very ill-conditioned (because of the high polynomial degree), but $\theta$ is very small which means we can fit the data very well.  Recall our formula for the condition number:

$$ \kappa(A) + \frac{\kappa(A)^2 \tan(\theta)}{\eta}. $$

In this case, because $\theta$ is small and $\eta$ is fairly large, both terms are comparable.  Thus we get a condition number of about $\kappa$, rather than $\kappa^2$.  This is often the case when the problem is ill-conditioned and the fit is close.

In [ ]:
print('least squares condition number: %0.2e' % (kappa + kappa**2*tantheta/eta))
print('kappa^2: %0.2e' % kappa**2)

Based on this, we should hope to get perhaps 6 digits of accuracy from a backward-stable algorithm (in double precision).  Meanwhile, the error from using the normal equations will be governed by $\kappa^2$, which is much larger.

# Solution methods and accuracy

## Numpy QR

First, let's try numpy's built-in QR factorization.

In [ ]:
Q, R = np.linalg.qr(A)
x = np.linalg.solve(R,np.dot(Q.T,b))
print(x[14]-1)

It does a bit better than what the analysis guarantees.  It seems this uses a backward-stable algorithm.

## Householder QR

Next, let's try using our own implementation of Householder factorization.

In [ ]:
def householder(A):
    """QR factorization via Householder triangularization."""
    m, n = A.shape
    V = np.zeros(A.shape)
    R = A.copy()
    for k in range(n-1):
        x = R[k:,k].copy()
        x[0] = x[0] + np.sign(x[0])*np.linalg.norm(x,2)
        x = x/np.linalg.norm(x,2)
        V[k:,k] = x.copy()
        for j in range(k,n):
            R[k:,j] = R[k:,j] - 2*V[k:,k]*np.dot(V[k:,k].T,R[k:,j])
    return V,R[:n,:]

def apply_Q(V,x):
    """Algorithm 10.3 of Trefethen & Bau."""
    m, n = V.shape
    for k in range(n-1,-1,-1):
        x[k:] = x[k:] - 2*np.dot(V[k:,k],x[k:])*V[k:,k]
    return x

def compute_Q(V):
    m, n = V.shape
    Q = np.zeros((m,n))
    for k in range(n):
        x = np.zeros(m)
        x[k] = 1.
        Q[:,k] = apply_Q(V,x)
    return Q

In [ ]:
V, R = householder(A)
Q = compute_Q(V)
x = np.linalg.solve(R,np.dot(Q.T,b))
print(x[14]-1)

This answer is totally wrong!  What happened?  Let's check that $Q$ is unitary:

In [ ]:
print(np.linalg.norm(np.dot(Q.T,Q)-np.eye(15)))

It is, so what's wrong?  Even though Householder is backward stable, my method for applying $Q^T$ is not.  However, there is a sneaky way to avoid this problem.  We just append $b$ to $A$ as a final column:

In [ ]:
M = np.append(A,b.reshape((m,1)),axis=1)
V, R = householder(M)
R2 = R[:n,:n]
Qb = R[:n,n]
x = np.linalg.solve(R2,Qb)
print(x[14]-1)

Now we get the expected accuracy -- roughly the same as numpy's QR.

### Modified Gram-Schmidt

Next, let's try Modified Gram-Schmidt.  We already know this is an unstable algorithm for computing $Q$, but here goes...

In [ ]:
def mgs(A):
    "QR factorization by the modified Gram-Schmidt algorithm."
    n = A.shape[1]
    R = np.zeros([n,n])
    V = np.zeros(A.shape)
    Q = np.zeros(A.shape)
    for i in range(n):
        V[:,i] = A[:,i]
    for i in range(n):
        R[i,i] = np.linalg.norm(V[:,i],2)
        Q[:,i] = V[:,i]/R[i,i]
        for j in range(i,n):
            R[i,j] = np.dot(Q[:,i].T,V[:,j])
            V[:,j] = V[:,j] - R[i,j]*Q[:,i]
    return Q, R

In [ ]:
Q, R = mgs(A)
x = np.linalg.solve(R,np.dot(Q.T,b))
print(x[14]-1)

In [ ]:
print(np.linalg.norm(np.dot(Q.T,Q)-np.eye(15)))

Since the $Q$ matrix we get from MGS is not orthogonal, then $Q^T$ is not its inverse, and the algorithm fails.  There is again a sneaky way to get around this and avoid the need to use $Q^T$ explicitly:

In [ ]:
M = np.append(A,b.reshape((m,1)),axis=1)
Q2, R = mgs(M)
R2 = R[:n,:n]
Qb = R[:n,n]
x = np.linalg.solve(R2,Qb)
print(x[14]-1)

Implemented this way, MGS is just as good as the other algorithms we've tried.

### Normal equations

What about the normal equations?

In [ ]:
x = np.linalg.solve(np.dot(A.T,A),np.dot(A.T,b))
print(x[14]-1)

Since $\kappa^2 \approx 10^{20}$, we don't get a single accurate digit from this algorithm.
However, the fact that $x$ is totally wrong doesn't necessarily mean $y$ is wrong.  Observe:

In [ ]:
y = np.dot(A,x)
plt.plot(t,b,'-',t,y,'o',lw=10);

Remember: **The fact that your model fits the data very well says absolutely nothing about whether the model is correct!**  

Also, if your least squares problem is ill-conditioned, then there can be vastly different models that fit the data almost equally well.  In this case it is a mistake to attach any meaning to the values of the model parameters.

### SVD

Finally, let's try the SVD:

In [ ]:
U, S, Vstar = np.linalg.svd(A)
x = np.dot(Vstar.T,np.dot(U[:,:n].T,b)/S)
print(x[14]-1)

This is almost exactly the same accuracy as we got from numpy's built-in QR function.

# Lower-degree fitting

Let's try fitting a more-oscillatory function with just a quadratic.

In [ ]:
m = 100  # Sample at this many points
n = 3   # Fit a polynomial of degree n-1

t = np.linspace(0,1,m)
A = np.zeros((m,n))
for i in range(n):
    A[:,i] = t**i      # Vandermonde matrix
    
b = np.exp(np.sin(40*t))  # Sampled function

x, res, rank, s = np.linalg.lstsq(A,b,rcond=-1)
y = np.dot(A,x)
kappa = np.linalg.cond(A)
eta = norm(A)*norm(x)/norm(y)
tantheta = norm(b-y)/norm(y)
term2 = kappa**2*tantheta/eta

print('kappa: %0.2e' % kappa)
print('tan(theta): %0.2e' % tantheta)
print('eta: %10.2e' % eta)
print('kappa^2 * tan(theta)/eta: %10.2e' % term2)
print('kappa^2: %0.2e' % kappa**2)
plt.plot(t,b,'-',t,y,'o');

# Use sympy to get "exact" solution
import sympy
def f(i,j):
    return sympy.Rational(i,m-1)**j
AA=sympy.Matrix(m,n,f)
bb = sympy.Matrix(b)
x_ex=AA.LDLsolve(bb)

# Note that we don't bother to normalize this time because x_ex[-1] is approximately 1 anyway.

As expected, the fit is not as good.  The value of $\theta$ is correspondingly much larger.  Of course, the smaller vandermonde matrix $A$ is much better conditioned than before.

Notice that now we expect the normal equations to be worse than a backward-stable algorithm by at most about a factor of 3.  Let's check:

## SVD

In [ ]:
U, S, Vstar = np.linalg.svd(A)
xx = np.dot(Vstar.T,np.dot(U[:,:n].T,b)/S)
print(xx[-1]-x_ex[-1])

## Normal equations

In [ ]:
xx = np.linalg.solve(np.dot(A.T,A),np.dot(A.T,b))
print(xx[-1]-x_ex[-1])

Indeed, for this case the normal equations give a result just as good as the SVD.